In [ ]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

In [ ]:
# Load test data
test_file = 'cylinder_wake.mat'
test_data = scipy.io.loadmat(f"./Data/{test_file}")

##### Bounds for testing ######
start_time = 140
end_time = 200


# Change to linspace/add slicing here
U_test = test_data['U_star'][...,start_time:end_time]  # 5000 x 2 x 200
P_test = test_data['p_star'][...,start_time:end_time]  # 5000 x 200
T_test = test_data['t'][start_time:end_time]  # 200 x 1
X_test = test_data['X_star']  # 5000 x 2

print(U_test.shape, P_test.shape, T_test.shape, X_test.shape)

N = X_test.shape[0]
T = T_test.shape[0]


# Rearrange Data
XX = np.tile(X_test[:, 0:1], (1, T))  # N x T
YY = np.tile(X_test[:, 1:2], (1, T))  # N x T
TT = np.tile(T_test, (1, N)).T  # N x T

UU = U_test[:, 0, :]  # N x T
VV = U_test[:, 1, :]  # N x T
PP = P_test  # N x T

x_test = XX.flatten()[:, None]  # NT x 1
y_test = YY.flatten()[:, None]  # NT x 1
t_test = TT.flatten()[:, None]  # NT x 1

u_test = UU.flatten()[:, None]  # NT x 1
v_test = VV.flatten()[:, None]  # NT x 1
p_test = PP.flatten()[:, None]  # NT x 1

# Convert test data to tensors
x = torch.tensor(x_test, dtype=torch.float32, requires_grad=True)
y = torch.tensor(y_test, dtype=torch.float32, requires_grad=True)
t = torch.tensor(t_test, dtype=torch.float32, requires_grad=True)
u = torch.tensor(u_test, dtype=torch.float32)
v = torch.tensor(v_test, dtype=torch.float32)
p = torch.tensor(p_test, dtype=torch.float32)

In [ ]:
nu = 0.01 # From very high Reynolds number
def create_pde(net, x, y, t):
    res = net(torch.hstack((x, y, t)))
    psi, p = res[:, 0:1], res[:, 1:2]

    u = torch.autograd.grad(psi, y, grad_outputs=torch.ones_like(psi), create_graph=True)[0]  
    v = -1. * torch.autograd.grad(psi, x, grad_outputs=torch.ones_like(psi), create_graph=True)[0]

    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]
    v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
    p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]

    f = u_t + u * u_x + v * u_y + p_x - nu * (u_xx + u_yy)
    g = v_t + u * v_x + v * v_y + p_y - nu * (v_xx + v_yy)

    return u, v, p, f, g

# Initialize the network
class ResNetBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ResNetBlock, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.activation = nn.Tanh()
    
    def forward(self, x):
        return self.activation(self.linear(x)) + x

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.input_layer = nn.Linear(3, 20)
        self.res_block1 = ResNetBlock(20, 20)
        self.res_block2 = ResNetBlock(20, 20)
        self.res_block3 = ResNetBlock(20, 20)
        self.res_block4 = ResNetBlock(20, 20)
        self.res_block5 = ResNetBlock(20, 20)
        self.res_block6 = ResNetBlock(20, 20)
        self.res_block7 = ResNetBlock(20, 20)
        self.output_layer = nn.Linear(20, 3)
        self.activation = nn.Tanh()
    
    def forward(self, x):
        x = self.activation(self.input_layer(x))
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = self.res_block5(x)
        x = self.res_block6(x)
        x = self.res_block7(x)
        x = self.output_layer(x)
        return x

net = ResNet()
state = torch.load('./models/pinn_skipped_timesteps_resnet_lbfgs.pt')
net.load_state_dict(state)
print("Model loaded.")

In [ ]:
# Null vector to test against f and g, and error func
null_vector = torch.zeros((x_test.shape[0], 1))
mse = nn.MSELoss()

# Get outputs
u_prediction, v_prediction, p_prediction, f_prediction, g_prediction = create_pde(net, x, y, t)
u_loss = mse(u_prediction, u)
v_loss = mse(v_prediction, v)
p_loss = mse(p_prediction, p)
f_loss = mse(f_prediction, null_vector)
g_loss = mse(g_prediction, null_vector)

loss = u_loss + v_loss + p_loss + f_loss + g_loss

# Reverse the manipulations done on input data
u_prediction_rs = np.reshape(u_prediction.detach().numpy(), UU.shape)
v_prediction_rs = np.reshape(v_prediction.detach().numpy(), VV.shape)
p_prediction_rs = np.reshape(p_prediction.detach().numpy(), PP.shape)

# Save to .mat for display
t_prediction_rs = T_test[start_time:end_time]
U_prediction_rs = np.stack([u_prediction_rs, v_prediction_rs], 1)

print(U_prediction_rs.shape, p_prediction_rs.shape)
scipy.io.savemat(f'./Results/run_{test_file}', {'U_star': U_prediction_rs, 'p_star': p_prediction_rs, 'X_star': X_test, 't': t_prediction_rs})

In [ ]:
print(u_loss, v_loss, p_loss)
print(f_loss, g_loss)